# Load Data

www.kaggle.com/moltean/fruits?select=fruits-360_dataset

In [1]:
import pandas as pd
import numpy as np
import os, zipfile, cv2, shutil, math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from PIL import Image
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm as tq
from tensorflow.keras.optimizers import Adam
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Kaggle/

Mounted at /content/drive
/content/drive/MyDrive/Kaggle


In [3]:
ls

fruits.zip


In [4]:
local_zip = '/content/drive/MyDrive/Kaggle/fruits.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Kaggle')
zip_ref.close()

In [5]:
os.listdir('/content/drive/MyDrive/Kaggle/fruits-360-original-size/fruits-360-original-size/Test')

['apple_6',
 'apple_braeburn_1',
 'apple_crimson_snow_1',
 'apple_golden_1',
 'apple_golden_2',
 'apple_golden_3',
 'apple_granny_smith_1',
 'apple_hit_1',
 'apple_pink_lady_1',
 'apple_red_1',
 'apple_red_2',
 'apple_red_3',
 'apple_red_delicios_1',
 'apple_red_yellow_1',
 'apple_rotten_1',
 'cabbage_white_1',
 'carrot_1',
 'cucumber_1',
 'cucumber_3',
 'eggplant_violet_1',
 'pear_1',
 'pear_3',
 'zucchini_1',
 'zucchini_dark_1']

In [7]:
base_dataset = '/content/drive/MyDrive/Kaggle/fruits-360-original-size/fruits-360-original-size/Test'
class_dir = ['apple_6',
 'apple_braeburn_1',
 'apple_crimson_snow_1',
 'apple_golden_1',
 'apple_golden_2',
 'apple_golden_3',
 'apple_granny_smith_1',
 'apple_hit_1',
 'apple_pink_lady_1',
 'apple_red_1',
 'apple_red_2',
 'apple_red_3',
 'apple_red_delicios_1',
 'apple_red_yellow_1',
 'apple_rotten_1',
 'cabbage_white_1',
 'carrot_1',
 'cucumber_1',
 'cucumber_3',
 'eggplant_violet_1',
 'pear_1',
 'pear_3',
 'zucchini_1',
 'zucchini_dark_1']
for class_item in class_dir:
  cur_dir = base_dataset + '/' + class_item + '/'
  dataset = os.listdir(cur_dir)
  for item in dataset :
    im = Image.open(f"{cur_dir}{item}")
    if item.endswith('.jpg'):
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
    else :
      im.save(f"{cur_dir}{item.split('.')[0]}.png", "png")
  for item in dataset :
    if not item.endswith('.png'):
      os.remove(os.path.join(cur_dir, item))

In [8]:
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(base_dataset):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [9]:
# membuat dataframe dari variabel baru
data = pd.DataFrame({'path':full_path,'file_name':file_name,'tag':tag})
print(data.groupby(['tag']).size())

tag
apple_6                 157
apple_braeburn_1        160
apple_crimson_snow_1    159
apple_golden_1          154
apple_golden_2          154
apple_golden_3          158
apple_granny_smith_1    160
apple_hit_1             234
apple_pink_lady_1       156
apple_red_1             154
apple_red_2             159
apple_red_3             140
apple_red_delicios_1    150
apple_red_yellow_1      154
apple_rotten_1          159
cabbage_white_1          47
carrot_1                 50
cucumber_1               50
cucumber_3               81
eggplant_violet_1        80
pear_1                  162
pear_3                   72
zucchini_1               80
zucchini_dark_1          80
dtype: int64


In [10]:
x = data['path']
y = data['tag']

In [11]:
X_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.30)

In [13]:
# menyatukan ke dalam masing-masing dataframe
data_tr = pd.DataFrame({'path':X_train, 'tag':y_train, 'set':'training'})

data_val = pd.DataFrame({'path':x_val, 'tag':y_val, 'set':'validation'})

In [14]:
print('train size', len(data_tr))
print('val size', len(data_val))

train size 2177
val size 933


In [15]:
dataset_path = '/content/drive/MyDrive/Kaggle/model_fruits'

data_all = data_tr.append([data_tr, data_val]).reset_index(drop=1)

for index, row in tq(data_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

0it [00:00, ?it/s]

In [16]:
os.listdir('/content/drive/MyDrive/Kaggle/model_fruits/training')

['apple_golden_3',
 'apple_crimson_snow_1',
 'apple_red_delicios_1',
 'apple_red_1',
 'pear_1',
 'apple_red_3',
 'apple_granny_smith_1',
 'apple_red_2',
 'apple_red_yellow_1',
 'apple_rotten_1',
 'apple_golden_1',
 'cabbage_white_1',
 'zucchini_dark_1',
 'apple_pink_lady_1',
 'apple_golden_2',
 'zucchini_1',
 'apple_6',
 'apple_hit_1',
 'cucumber_1',
 'pear_3',
 'eggplant_violet_1',
 'carrot_1',
 'cucumber_3',
 'apple_braeburn_1']

# Preprocessing

In [17]:
# Gather data train
train_data = []
train_label = []

train_dir='/content/drive/MyDrive/Kaggle/model_fruits/training'
for r, d, f in os.walk(train_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            train_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            train_label.append(label)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [18]:
# Gather data validation
val_data = []
val_label = []

vali_dir='/content/drive/MyDrive/Kaggle/model_fruits/validation'
for r, d, f in os.walk(vali_dir):
    for file in f:
        if ".png" in file:
            imagePath = os.path.join(r, file)
            image = cv2.imread(imagePath)
            image = cv2.resize(image, (250,250))
            val_data.append(image)
            label = imagePath.split(os.path.sep)[-2]
            val_label.append(label)

val_data = np.array(val_data)
val_label = np.array(val_label)

In [19]:
print('Data Training Shape : ', train_data.shape)
print('Data Validation Shape : ', val_data.shape)

Data Training Shape :  (2177, 250, 250, 3)
Data Validation Shape :  (933, 250, 250, 3)


In [20]:
# Normalisasi dataset
print("Data sebelum di-normalisasi ", train_data[0][0][0])

X_train = train_data.astype('float32') / 255.0
x_val = val_data.astype('float32') / 255.0
print("Data setelah di-normalisasi ", X_train[0][0][0])

Data sebelum di-normalisasi  [255 255 255]
Data setelah di-normalisasi  [1. 1. 1.]


In [21]:
# Transformasi label encoder
print('Label sebelum di-encoder ', train_label[2166:2176])

lb = LabelEncoder()
y_train = lb.fit_transform(train_label)
y_val = lb.fit_transform(val_label)

print('Label setelah di-encoder ', y_train[2166:2176])

Label sebelum di-encoder  ['apple_braeburn_1' 'apple_braeburn_1' 'apple_braeburn_1'
 'apple_braeburn_1' 'apple_braeburn_1' 'apple_braeburn_1'
 'apple_braeburn_1' 'apple_braeburn_1' 'apple_braeburn_1'
 'apple_braeburn_1']
Label setelah di-encoder  [1 1 1 1 1 1 1 1 1 1]


# Modelling

In [22]:
model = Sequential()

model.add(InputLayer(input_shape=[250,250,3]))
model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(GlobalMaxPooling2D())
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Print model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [24]:
model2 = Sequential()

model2.add(InputLayer(input_shape=[250,250,3]))
model2.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model2.add(AveragePooling2D(pool_size=2, padding='same'))
model2.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu'))
model2.add(AveragePooling2D(pool_size=2, padding='same'))
model2.add(Conv2D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu'))
model2.add(GlobalAveragePooling2D())
model2.add(Flatten())

# Fully Connected Layer
model2.add(Dense(512, activation='relu'))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# Print model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 125, 125, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0

In [25]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

In [27]:
H = model.fit(
    X_train, 
    y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(x_val, y_val)
    )

Epoch 1/100
69/69 [==============================] - 38s 97ms/step - loss: nan - acc: 0.0524 - val_loss: nan - val_acc: 0.0472
Epoch 2/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 3/100
69/69 [==============================] - 6s 80ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 4/100
69/69 [==============================] - 5s 80ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 5/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 6/100
69/69 [==============================] - 5s 80ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 7/100
69/69 [==============================] - 6s 81ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 8/100
69/69 [==============================] - 6s 80ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc

In [28]:
model2.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

H2 = model.fit(
    X_train, 
    y_train, 
    batch_size=32, 
    epochs=100, 
    validation_data=(x_val, y_val)
    )

Epoch 1/100
69/69 [==============================] - 6s 92ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 2/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 3/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 4/100
69/69 [==============================] - 6s 81ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 5/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 6/100
69/69 [==============================] - 6s 81ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 7/100
69/69 [==============================] - 5s 79ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc: 0.0472
Epoch 8/100
69/69 [==============================] - 5s 80ms/step - loss: nan - acc: 0.0519 - val_loss: nan - val_acc:

In [29]:
from sklearn.metrics import classification_report

pred = model.predict(x_val)
labels = (pred > 0.5).astype(np.int)
print(classification_report(y_val, labels))

              precision    recall  f1-score   support

           0       0.05      1.00      0.09        44
           1       0.00      0.00      0.00        57
           2       0.00      0.00      0.00        42
           3       0.00      0.00      0.00        43
           4       0.00      0.00      0.00        40
           5       0.00      0.00      0.00        51
           6       0.00      0.00      0.00        43
           7       0.00      0.00      0.00        65
           8       0.00      0.00      0.00        51
           9       0.00      0.00      0.00        43
          10       0.00      0.00      0.00        44
          11       0.00      0.00      0.00        43
          12       0.00      0.00      0.00        41
          13       0.00      0.00      0.00        44
          14       0.00      0.00      0.00        52
          15       0.00      0.00      0.00        12
          16       0.00      0.00      0.00        18
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
import pickle
model.save('model.h5')

In [32]:
model2.save('model.h5')